In [1]:
import numpy as np
import pandas as pd
import re
import random
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_csv("./data/query_pattern_data.csv")
df.head()

,Query Pattern,Intent,Attribute,Aggregation,Paid Status
0,Who are my outstanding debtors?,Invoice,None,List,FALSE
1,What are the names of my outstanding debtors?,Invoice,None,List,FALSE
2,Who are my outstanding creditors?,Invoice,None,List,FALSE
3,What are the names of my outstanding creditors?,Invoice,None,List,FALSE
4,What was the [aggregation_entity] invoice amou...,Sales,Customer,NaN,None


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df["Query Pattern"], df["Attribute"], test_size = 0.2)

df.loc[X_train.index, "split"] = "train"
df.loc[X_test.index, "split"] = "test"

In [4]:
customers = pd.read_csv("data/customers.csv")
products = pd.read_csv("data/products.csv")
vendors = pd.read_csv("data/vendors.csv")
accounts = pd.read_csv("data/chart_of_accounts.csv")
employee = pd.read_csv("data/employees.csv")
date_ranges = pd.read_csv("data/date_ranges.csv")

ignore_keys = ["sample", "test", "quickbooks", "unknown", "my company", "customer", "cash", "sale", "deposit", ".", "payroll", "anonymous"]

In [5]:
def clean_names(df, var):
    flag = df[var].apply(lambda x: sum([y in x for y in ignore_keys]))
    df = df[flag == 0].reset_index(drop = True)
    return df

In [6]:
def stem_tokens(x):
    return " ".join([lemmatizer.lemmatize(y) for y in x.lower().split(" ")])

In [7]:
def get_variants(x):
    variants = [x.lower()]
    variants.append(stem_tokens(x))
    if "&" in x:
        x0 = x.split("&")[0].strip().lower()
        x1 = x.split("&")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    if "|" in x:
        x0 = x.split("|")[0].strip().lower()
        x1 = x.split("|")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    variants = np.unique(variants)
    return variants

In [8]:
def choose_random_attribute(filter_by_attribute):
    return random.sample(filter_by_attribute[random.sample(sorted(filter_by_attribute), 1)[0]].tolist(), 1)[0]

In [14]:
customers = clean_names(customers, "Customer name")
products = clean_names(products, "Product/Service")
vendors = clean_names(vendors, "Vendor name")
accounts = clean_names(accounts, "Account name")
employee = clean_names(employee, "Employee name")

In [19]:
customers["Variants"] = customers["Customer name"].apply(lambda x: get_variants(x))
products["Variants"] = products["Product/Service"].apply(lambda x: get_variants(x))
vendors["Variants"] = vendors["Vendor name"].apply(lambda x: get_variants(x))
accounts["Variants"] = accounts["Account name"].apply(lambda x: get_variants(x))
employee["Variants"] = employee["Employee name"].apply(lambda x: get_variants(x))

In [22]:
K = 50

aggregation_entity = ["total", "average", "mean", "max", "min", "first", "last", "highest", "lowest"]

intents = ["sales", "expense", "invoice", "bill", "account payable", "account receivable"]

date_filter_entity = date_ranges["sample date ranges in questions"].tolist()

filter_by_customer = customers[:K].set_index("Customer name").to_dict()["Variants"]

filter_by_product = products[:K].set_index("Product/Service").to_dict()["Variants"]

filter_by_vendor = vendors[:K].set_index("Vendor name").to_dict()["Variants"]

filter_by_account = accounts[:K].set_index("Account name").to_dict()["Variants"]

filter_by_employee = employee[:K].set_index("Employee name").to_dict()["Variants"]

group_by_entity = ["account", "department", "vendor", "customer", "product"]

In [23]:
attributes_list = df["Query Pattern"].apply(lambda x:  re.findall(r'\[.*?\]', x))
attributes_list = attributes_list.apply(lambda x: [y.replace("[", "").replace("]", "") for y in x])
attributes_list = [x for y in attributes_list for x in y]
pd.Series(attributes_list).value_counts()

date_filter           67
customer_name         44
aggregation_entity    14
product_name          10
vendor_name           10
salesperson_name       9
account_name           9
groupby_entity         1
dtype: int64

In [24]:
attributes_dict = {
    "date_filter": date_filter_entity,
    "customer_name": filter_by_customer,
    "vendor_name": filter_by_vendor,
    "product_name": filter_by_product,
    "aggregation_entity": aggregation_entity,
    "account_name": filter_by_account,
    "employee_name": filter_by_employee,
    "groupby_entity": group_by_entity
}

In [25]:
def choose_random_attribute(attribute_dict):
    return random.sample(attribute_dict[random.sample(sorted(attribute_dict), 1)[0]].tolist(), 1)[0]

In [26]:
def replace_attributes(query_pattern):
    attributes = re.findall(r'\[.*?\]', query_pattern)
    for a in attributes:
        a_cleaned = re.sub(r"[\[\]]", "", a)
        if type(attributes_dict[a_cleaned]) == list:
            select_attribute = random.sample(attributes_dict[a_cleaned], 1)[0]
        else:
            select_attribute = choose_random_attribute(attributes_dict[a_cleaned])
        query_pattern = query_pattern.replace(a, select_attribute)
        
    return query_pattern

In [27]:
def get_aggregation(query):
    if any(word in query for word in ["average", "mean"]):
        return "Average"
    elif any(word in query for word in ["max", "last", "highest"]):
        return "Max"
    elif any(word in query for word in ["min", "first", "lowest"]):
        return "Min"
    else:
        return "Total"

In [28]:
def simulate_questions(n):
    df_sample = df[~df["Query Pattern"].apply(lambda x: "salesperson_name" in x)].reset_index(drop=True)
    df_simulated = df_sample.loc[random.choices(range(len(df_sample)), k=n)].reset_index(drop=True)
    df_simulated["Query"] = df_simulated["Query Pattern"].apply(lambda x: replace_attributes(x))
    df_simulated.loc[df_simulated["Aggregation"].isna(), "Aggregation"] = df_simulated[df_simulated["Aggregation"].isna()]["Query"].apply(lambda x: get_aggregation(x))
    return df_simulated

In [29]:
df_simulated = simulate_questions(5000)

In [30]:
df_simulated[["Query", "Intent", "Aggregation", "Paid Status", "split"]].to_csv("./df_simulated_v2.csv", index=False)

In [31]:
df_simulated.head()

,Query Pattern,Intent,Attribute,Aggregation,Paid Status,split,Query
0,How many invoices have we sent to [customer_na...,Invoice,Customer,Count,None,test,How many invoices have we sent to freeman spor...
1,"[date_filter], how many [product_name] did I s...",Invoice,Product,Count,None,train,"This quarter, how many allowance did I sell?"
2,Who are my outstanding debtors?,Invoice,None,List,FALSE,train,Who are my outstanding debtors?
3,"[date_filter], how many invoices were paid?",Invoice,None,Count,TRUE,train,"Last month, how many invoices were paid?"
4,What was our biggest expense [date_filter]?,Expense,None,Max,None,train,What was our biggest expense Last quarter?
